## Expressions 

**Expression** 提供了一个通用的结构既能够解决简单的查询，又能轻松扩展到复杂的查询.

- select
- filter
- with_columns
- group_by



### Select statement 

为了能选择一列 
1. 定义 `DataFrame`
2. 选择数据


In [44]:
import polars as pl 
from datetime import datetime 

In [45]:
df = pl.DataFrame(
   {
        "integer": [1, 2, 3],
        "date": [
            datetime(2022, 1, 1),
            datetime(2022, 1, 2),
            datetime(2022, 1, 3),
        ],
        "float": [4.0, 5.0, 6.0],
    }
)
print(df)
    

shape: (3, 3)
┌─────────┬─────────────────────┬───────┐
│ integer ┆ date                ┆ float │
│ ---     ┆ ---                 ┆ ---   │
│ i64     ┆ datetime[μs]        ┆ f64   │
╞═════════╪═════════════════════╪═══════╡
│ 1       ┆ 2022-01-01 00:00:00 ┆ 4.0   │
│ 2       ┆ 2022-01-02 00:00:00 ┆ 5.0   │
│ 3       ┆ 2022-01-03 00:00:00 ┆ 6.0   │
└─────────┴─────────────────────┴───────┘


In [46]:
# 选择所有列
df.select(pl.col("*")) 
# 选择某一列输出 
df.select(pl.col("integer"))


integer
i64
1
2
3


In [56]:
import pandas as pd
from io import StringIO

# 输入数据
data = """a   b        c                   d
0   0.718233 2022-12-01 00:00:00  1.0
1   0.381739 2022-12-02 00:00:00  2.0
2   0.384342 2022-12-03 00:00:00  NaN
3   0.847523 2022-12-04 00:00:00  NaN
4   0.531582 2022-12-05 00:00:00  0.0
5   0.5351   2022-12-06 00:00:00 -5.0
6   0.479497 2022-12-07 00:00:00 -42.0
7   0.770922 2022-12-08 00:00:00 NaN
"""

# 读取数据到Pandas DataFrame
pandas_df = pd.read_csv(StringIO(data), delim_whitespace=True)

# 将连在一起的日期时间字符串分割成日期和时间两列
# pandas_df[['c_date', 'c_time']] = pandas_df['c'].str.split(' ', 1, expand=True)

# 转换为Polars DataFrame
df2 = pl.DataFrame(pandas_df)

# 打印DataFrame
print(df2)

shape: (8, 4)
┌──────────┬────────────┬──────────┬───────┐
│ a        ┆ b          ┆ c        ┆ d     │
│ ---      ┆ ---        ┆ ---      ┆ ---   │
│ f64      ┆ str        ┆ str      ┆ f64   │
╞══════════╪════════════╪══════════╪═══════╡
│ 0.718233 ┆ 2022-12-01 ┆ 00:00:00 ┆ 1.0   │
│ 0.381739 ┆ 2022-12-02 ┆ 00:00:00 ┆ 2.0   │
│ 0.384342 ┆ 2022-12-03 ┆ 00:00:00 ┆ null  │
│ 0.847523 ┆ 2022-12-04 ┆ 00:00:00 ┆ null  │
│ 0.531582 ┆ 2022-12-05 ┆ 00:00:00 ┆ 0.0   │
│ 0.5351   ┆ 2022-12-06 ┆ 00:00:00 ┆ -5.0  │
│ 0.479497 ┆ 2022-12-07 ┆ 00:00:00 ┆ -42.0 │
│ 0.770922 ┆ 2022-12-08 ┆ 00:00:00 ┆ null  │
└──────────┴────────────┴──────────┴───────┘


**选择几列数据**

In [ ]:
df2.select(pl.col("a", "c","b"))

In [ ]:
# df2.select(pl.col("a"),pl.col("b")).limit(5)
df2.select(pl.col("a","b")).limit(5)

**使用 `pl.exclude()` 排除某些列**

In [48]:
exculde_list = ["a" ,"c"] # 设置排除的列表项 

df2.select(pl.exclude(exculde_list))

b,d
str,f64
"""2022-12-01""",1.0
"""2022-12-02""",2.0
"""2022-12-03""",null
"""2022-12-04""",null
"""2022-12-05""",0.0
"""2022-12-06""",-5.0
"""2022-12-07""",-42.0
"""2022-12-08""",null


#### Filter 过滤
过滤器选项允许我们创建DataFrame的一个子集。我们使用与前面相同的DataFrame，并在两个指定日期之间进行筛选。

In [55]:
# df2.filter(
#     pl.col("c").is_between(datetime(2022, 12, 2), datetime(2022, 12, 8)),
# )
print(df2)
# 过滤出a列数值在 [0.4 - 0.6]的行
# df2 = df2.filter(

#     pl.col("a").is_between(0.4,0.6),
# )


df2 = df2.filter((pl.col("a") <= 0.5) & (pl.col("d").is_not_null()))
print(df2)

shape: (3, 4)
┌──────────┬────────────┬──────────┬───────┐
│ a        ┆ b          ┆ c        ┆ d     │
│ ---      ┆ ---        ┆ ---      ┆ ---   │
│ f64      ┆ str        ┆ str      ┆ f64   │
╞══════════╪════════════╪══════════╪═══════╡
│ 0.381739 ┆ 2022-12-02 ┆ 00:00:00 ┆ 2.0   │
│ 0.384342 ┆ 2022-12-03 ┆ 00:00:00 ┆ null  │
│ 0.479497 ┆ 2022-12-07 ┆ 00:00:00 ┆ -42.0 │
└──────────┴────────────┴──────────┴───────┘
shape: (2, 4)
┌──────────┬────────────┬──────────┬───────┐
│ a        ┆ b          ┆ c        ┆ d     │
│ ---      ┆ ---        ┆ ---      ┆ ---   │
│ f64      ┆ str        ┆ str      ┆ f64   │
╞══════════╪════════════╪══════════╪═══════╡
│ 0.381739 ┆ 2022-12-02 ┆ 00:00:00 ┆ 2.0   │
│ 0.479497 ┆ 2022-12-07 ┆ 00:00:00 ┆ -42.0 │
└──────────┴────────────┴──────────┴───────┘


### With_columns 

创建新列

In [67]:
df2.with_columns(pl.col("a").sum().alias("e"), (pl.col("a") + 42).alias("b+42"))
# with_columns(新列1,新列2)
# alias 命名

a,b,c,d,e,b+42
f64,str,str,f64,f64,f64
0.718233,"""2022-12-01""","""00:00:00""",1.0,4.648938,42.718233
0.381739,"""2022-12-02""","""00:00:00""",2.0,4.648938,42.381739
0.384342,"""2022-12-03""","""00:00:00""",null,4.648938,42.384342
0.847523,"""2022-12-04""","""00:00:00""",null,4.648938,42.847523
0.531582,"""2022-12-05""","""00:00:00""",0.0,4.648938,42.531582
0.5351,"""2022-12-06""","""00:00:00""",-5.0,4.648938,42.5351
0.479497,"""2022-12-07""","""00:00:00""",-42.0,4.648938,42.479497
0.770922,"""2022-12-08""","""00:00:00""",null,4.648938,42.770922


In [68]:
df2.with_columns( pl.col("b")+pl.col("c").alias("f"))
df2

a,b,c,d
f64,str,str,f64
0.718233,"""2022-12-01""","""00:00:00""",1.0
0.381739,"""2022-12-02""","""00:00:00""",2.0
0.384342,"""2022-12-03""","""00:00:00""",null
0.847523,"""2022-12-04""","""00:00:00""",null
0.531582,"""2022-12-05""","""00:00:00""",0.0
0.5351,"""2022-12-06""","""00:00:00""",-5.0
0.479497,"""2022-12-07""","""00:00:00""",-42.0
0.770922,"""2022-12-08""","""00:00:00""",null


### Group by

In [69]:
df3 = pl.DataFrame(
    {
        "x": range(8),
        "y": ["A", "A", "A", "B", "B", "C", "X", "X"],
    }
)

In [70]:
df3

x,y
i64,str
0,"""A"""
1,"""A"""
2,"""A"""
3,"""B"""
4,"""B"""
5,"""C"""
6,"""X"""
7,"""X"""


In [74]:
df3.group_by("y", maintain_order=True).count()

y,count
str,u32
"""A""",3
"""B""",2
"""C""",1
"""X""",2


In [75]:
df3.group_by("y", maintain_order=True).agg(
    pl.col("*").count().alias("count"),
    pl.col("*").sum().alias("sum"),
)

y,count,sum
str,u32,i64
"""A""",3,3
"""B""",2,7
"""C""",1,5
"""X""",2,13


### Combining operations

In [88]:
df2
# df_x = df2.with_columns(( (pl.col("a") + pl.col("d").is_not_null() )& (pl.col("d"))).alias("a* d"))

a,b,c,d
f64,str,str,f64
0.718233,"""2022-12-01""","""00:00:00""",1.0
0.381739,"""2022-12-02""","""00:00:00""",2.0
0.384342,"""2022-12-03""","""00:00:00""",null
0.847523,"""2022-12-04""","""00:00:00""",null
0.531582,"""2022-12-05""","""00:00:00""",0.0
0.5351,"""2022-12-06""","""00:00:00""",-5.0
0.479497,"""2022-12-07""","""00:00:00""",-42.0
0.770922,"""2022-12-08""","""00:00:00""",null
